In [1]:
from pypdf import PdfReader

In [2]:
def extract_text_from_pdf(pdf_path):
    # Path to the PDF file
    pdf_path = 'data/Finance Project (Final).pdf'

    # Create a PdfReader object
    reader = PdfReader(pdf_path)

    # Initialize a string to hold all the extracted text
    extracted_text = ""

    # Loop through each page and extract text
    for page in reader.pages:
        extracted_text += page.extract_text()

    # Print the extracted text
    # with open('extracted_text.txt', 'w') as text_file:
        # text_file.write(extracted_text)

    return extracted_text

In [3]:
pdf_path = 'data/Finance Project (Final).pdf'
text = extract_text_from_pdf(pdf_path)

In [4]:
len(text)

116975

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from langchain_community.document_loaders import TextLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True,
)

documents = []

loader = TextLoader(os.path.join('./', 'extracted_text.txt'), encoding="utf-8")
documents.extend(loader.load())

chunks = text_splitter.split_documents(documents)
print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

Split 1 documents into 371 chunks.


In [9]:
# from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer('all-MiniLM-L6-v2')

In [20]:
from langchain_community.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2", model_kwargs={"trust_remote_code":True}) 

doc_result = embeddings.embed_documents(["text"])
print(doc_result)
print(len(doc_result))

[[0.014475906267762184, 0.07228005677461624, -0.021847398951649666, 0.027838347479701042, -0.032556332647800446, 0.03891464322805405, 0.13196884095668793, 0.04014265537261963, 0.11188974976539612, -0.04175683110952377, 0.04023201763629913, 0.057262271642684937, 0.014718607068061829, -0.002374015748500824, 0.000651488546282053, 0.01911281980574131, 0.07063929736614227, -0.04581274092197418, -0.08963267505168915, -0.053711023181676865, 0.019062379375100136, 0.07670910656452179, 0.03966069966554642, 0.041070178151130676, -0.002009959891438484, 0.12502677738666534, -0.1073010042309761, 0.06206385791301727, 0.04361991956830025, -0.04085909575223923, -0.04527280852198601, 0.03711407259106636, 0.14580844342708588, 0.03468615189194679, -0.04757186397910118, 0.009284678846597672, -0.026461714878678322, 0.027106789872050285, -0.01844361238181591, 0.00240807025693357, -0.031192444264888763, -0.10679488629102707, 0.004846959374845028, 0.01772296614944935, 0.07387688010931015, -0.02600938268005848,

In [21]:
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(
    chunks, embeddings, persist_directory="./chroma2"
)

In [29]:
def retrieve_relevant_chunks(query):
    result = db.similarity_search_with_relevance_scores(query=query, k=10)
    return result

In [23]:
# Step 6: Generate questions
def generate_questions(context):
    question_generator = pipeline("question-generation")
    questions = question_generator(context)
    return questions

In [30]:
query = "What are the main points of the document?"
relevant_chunks = retrieve_relevant_chunks(query)
print(relevant_chunks)

[(Document(metadata={'source': './extracted_text.txt', 'start_index': 1112}, page_content='6. Comparative  Volatility  Analysis  across  Time  Zones ………………………………... 3 3 \n7. Recommendations  and Implications  .............................................................................. 35 \n8. Conclusion  ........................................................................................................................ 37 \n9. Bibliography .................................................................................................................... 39'), -0.0417235927438766), (Document(metadata={'source': './extracted_text.txt', 'start_index': 241}, page_content='5. Parth Kushwaha  : 23103064  2  Table  of Contents  \n1. Abstract  ...............................................................................................................................3  \n2. Introduction  ..................................................................................................

C:\Users\bhavy\AppData\Local\Temp\ipykernel_5740\2948955076.py:2: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'source': './extracted_text.txt', 'start_index': 1112}, page_content='6. Comparative  Volatility  Analysis  across  Time  Zones ………………………………... 3 3 \n7. Recommendations  and Implications  .............................................................................. 35 \n8. Conclusion  ........................................................................................................................ 37 \n9. Bibliography .................................................................................................................... 39'), -0.0417235927438766), (Document(metadata={'source': './extracted_text.txt', 'start_index': 241}, page_content='5. Parth Kushwaha  : 23103064  2  Table  of Contents  \n1. Abstract  ..........................................................................................................................

In [26]:
# Generate questions
context = " ".join(relevant_chunks)
questions = generate_questions(context)
for q in questions:
    print(q['question'])

TypeError: sequence item 0: expected str instance, tuple found

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
import chromadb

# Define the embedding function using SentenceTransformerEmbeddings
def get_embedding(text):
    # Initialize the sentence transformer with model_kwargs
    embeddings_model = SentenceTransformerEmbeddings(
        model_name="all-MiniLM-L6-v2",
        model_kwargs={"trust_remote_code": True}
    )
    
    # Get the embeddings for the given text
    embeddings = embeddings_model.embed([text])  # The embed method expects a list of texts
    return embeddings[0]  # Return the first embedding (as we are embedding one document at a time)

# Initialize Chroma client with the new embedding function
client = chromadb.Client()

collection = client.create_collection(
    name="pdf_collection", 
    embedding_function=get_embedding  # Embedding function using SentenceTransformer
)

# Example of adding a document with the new embeddings
collection.add(
    documents=["This is the content of the PDF."],  
    ids=["pdf1"],
    metadatas=[{"name": "PDF1"}]
)

In [52]:
import os
import chromadb
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader

# Initialize the ChromaDB client
client = chromadb.Client()
def get_embedding(text):
    # Initialize the sentence transformer with model_kwargs
    embeddings_model = SentenceTransformerEmbeddings(
        model_name="all-MiniLM-L6-v2",
        model_kwargs={"trust_remote_code": True}
    )
    
    # Get the embeddings for the given text
    embeddings = embeddings_model.embed([text])  # The embed method expects a list of texts
    return embeddings[0]  # Return the first embedding (as we are embedding one document at a time)

# Create a collection for storing PDF embeddings
collection_name = "pdf_embeddings"
# collection = client.create_collection(name=collection_name)
collection = client.create_collection(
    name="pdf_collection1", 
    embedding_function=get_embedding  # Embedding function using SentenceTransformer
)

ValueError: Expected EmbeddingFunction.__call__ to have the following signature: odict_keys(['self', 'input']), got odict_keys(['self', 'args', 'kwargs'])
Please see https://docs.trychroma.com/guides/embeddings for details of the EmbeddingFunction interface.
Please note the recent change to the EmbeddingFunction interface: https://docs.trychroma.com/deployment/migration#migration-to-0.4.16---november-7,-2023 


In [44]:
# Load the embedding model
model_name = "all-MiniLM-L6-v2"
model = SentenceTransformer(model_name, model_kwargs={"trust_remote_code": True})

# Function to process a PDF, generate embeddings, and store them in ChromaDB
def store_pdf_embeddings(pdf_file_path, doc_id):
    # Extract text from the PDF
    text = extract_text_from_pdf(pdf_file_path)

    # Generate embeddings
    embeddings = model.encode(text)

    # Store embeddings in the ChromaDB collection
    collection.add(
        documents=[text],  # Store document text
        embeddings=[embeddings],  # Store the corresponding embeddings
        ids=[doc_id]  # Unique ID for the PDF
    )
    print(f"Embeddings for {pdf_file_path} stored successfully!")

# Example usage
pdf_file_path = "data/Finance Project (Final).pdf"  # Replace with your PDF file path
doc_id = "pdf_1"  # Unique identifier for the PDF document
store_pdf_embeddings(pdf_file_path, doc_id)
store_pdf_embeddings("data/Resume (Digital).pdf", "pdf_2")

Add of existing embedding ID: pdf_1
Insert of existing embedding ID: pdf_1


Embeddings for data/Finance Project (Final).pdf stored successfully!
Embeddings for data/Resume (Digital).pdf stored successfully!


In [45]:
def get_pdf_embeddings(doc_id):
    result = collection.get(ids=[doc_id])
    if result["embeddings"]:
        print(f"Embeddings for doc_id '{doc_id}' retrieved successfully!")
        return result["embeddings"][0]
    else:
        print(f"No embeddings found for doc_id '{doc_id}'")
        return None


In [49]:
store_pdf_embeddings("data/Resume (Digital).pdf", "pdf_3")
get_pdf_embeddings("pdf_1")

Add of existing embedding ID: pdf_3
Insert of existing embedding ID: pdf_3


Embeddings for data/Resume (Digital).pdf stored successfully!
No embeddings found for doc_id 'pdf_1'
